## Day 49 Lecture 1 Assignment

In this assignment, we will apply GMM (Gaussian Mixture Modeling) clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [13]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.special import entr

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [14]:
# answer goes here
url = 'https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv'

df = pd.read_csv(url)




The goal is to cluster these player-seasons to identify potential player "archetypes".  
The pre-processing steps will be identical to what we previously did for K-means.

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [15]:
# answer goes here

df = df[df['GS'] >=20]
df = df[df['MP'] >= 10]



Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [16]:
# answer goes here
X = df[['PTS','TRB', 'AST', 'STL', 'BLK', 'TOV']]





Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [17]:
# answer goes here
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

Run both K-Means and Gaussian mixtures modeling twice, once with 3 cluster and once with 7 clusters. Print out the resulting means for all 4 scenarios (KM+3, GMM+3, KM+7, GMM+7). When printing the means, transform the scaled versions back into their corresponding unscaled values. 

What "archetypes" do you see? Are the archetypes identified by GMM similar to those identified by K-Means? How do the means of GMM differ from those of K-Means?

KM

In [18]:
# answer goes here
km3 = KMeans(n_clusters=3)

X['kms3'] = km3.fit_predict(X_scaled)

#centers = scaler.inverse_transform(km3.cluster_centers_)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [19]:
center_km3 = pd.DataFrame(km3.cluster_centers_, columns=X_scaled.columns)

center_km3

,PTS,TRB,AST,STL,BLK,TOV
0,-0.468976,-0.517071,-0.396048,-0.279474,-0.439358,-0.539769
1,0.086332,1.269641,-0.467217,-0.345349,1.289195,-0.014648
2,1.075975,-0.001260,1.454813,1.042428,-0.213796,1.353680


In [20]:
km7 = KMeans(n_clusters=7)

X['kms7'] = km7.fit_predict(X_scaled)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
center_km7 = pd.DataFrame(km7.cluster_centers_, columns=X_scaled.columns)

center_km7

,PTS,TRB,AST,STL,BLK,TOV
0,0.882207,-0.254544,1.362077,0.912057,-0.405003,1.103551
1,-0.267983,0.803563,-0.558781,-0.642600,0.712597,-0.314931
2,-0.994228,-0.671026,-0.743648,-0.786801,-0.442788,-0.985116
3,0.116120,-0.613146,0.079172,-0.200980,-0.634691,-0.007130
4,1.902653,1.046034,2.225524,1.743783,0.402855,2.637334
5,0.779215,2.001306,-0.287186,0.011949,2.380170,0.558333
6,-0.316373,-0.117155,-0.393721,1.063120,-0.166812,-0.483715


GMM

In [22]:
gmm3 = GaussianMixture(n_components=3, covariance_type='full')
gmm3.fit(X_scaled)
X['gmm_clus_3'] = gmm3.predict(X_scaled)
probs = pd.DataFrame(gmm3.predict_proba(X_scaled))
X = pd.concat([X, probs], axis=1)
X.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,PTS,TRB,AST,STL,BLK,TOV,kms3,kms7,gmm_clus_3,0,1,2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.731826,5.687710e-04,0.267606
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.578037,2.286479e-03,0.419676
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000440,4.904223e-08,0.999560
3,13.9,9.5,1.6,1.5,1.0,1.7,1.0,6.0,0.0,0.002115,1.252430e-09,0.997885
4,8.9,7.3,2.2,0.9,0.8,1.5,1.0,1.0,0.0,0.870642,1.307117e-04,0.129227


Predict the likelihood of each player belonging to one of the 3 clusters using the GMM model. Then, calculate the entropy for each set of predicted probabilities. 

We will use entropy as a measure of how confident we are in the predicted class label. If we had no confidence in our prediction, we would assign 33% probability to each class, while if we were totally confident, we would assign 100% to one class. Entropy would be at a maximum in the "no confidence" scenario and a minimum in the "full confidence" scenario, which makes it a reasonable way to quantify our uncertainty in our prediction. There are certainly other methods as well; feel free to experiment with them if desired.

Which five predicted labels are we least confident about? Which five are we most confident about? Print out the associated details (season, player name, stats, etc.) from those players.

In [ ]:
# answer goes here



